In [1]:
import numpy as np
import scipy.stats as stats
from math import sqrt

In [2]:
def check_H_0(p_value, alpha):
    if p_value <= alpha:
        print('Reject H_0')
    else:
        print('Accept H_0')

In [3]:
ALPHA = 0.05

DATA_SOURCE_PATH_7_14 = 'data_samples/T3_1_HEIGHTWT.DAT'
DATA_SOURCE_PATH_7_15 = 'data_samples/T6_12_CALCSPD.DAT'
DATA_SOURCE_PATH_7_28 = 'data_samples/T3_7_SONS.DAT'


## 7.14

In Example 5.2.2, we assumed that for the height and weight data of Table 3.1,
the population covariance matrix is:

$\sum = \left( \begin{array}{cc} 20 & 100 \\ 100 & 1000 \end{array} \right)$

Test this as a hypothesis using (7.2).

In [4]:
E_0 = np.asarray([[20, 100], [100, 1000]])


In [5]:
def load_data_7_14(data_source_path):
    data = np.loadtxt(data_source_path, dtype=int)[:, 1:]

    nu, p = data.shape

    S = np.cov(data, rowvar=False)
    eig_vals, _ = np.linalg.eig(S @ np.linalg.pinv(E_0))
    eig_vals = eig_vals[:p]

    u = nu * (np.sum(eig_vals - np.log(eig_vals)) - p)
    u_t = (1 - (1 / (6 * nu - 1) * (2 * p + 1 - (2 / (p + 1))))) * u

    df = (0.5 * p * (p + 1))
    xi_critical = stats.chi2.ppf(1 - ALPHA, df=df)

    return u_t, xi_critical

In [6]:
u_t, xi_critical = load_data_7_14(DATA_SOURCE_PATH_7_14)

In [7]:
check_H_0(u_t, xi_critical)

Accept H_0


## 7.15

$H_{0} : \sum = σ^2I$

and

$H_{0} : C \sum C^{'} = σ^2I$

for the calculator speed data of Table 6.12.

In [8]:
def load_data_7_15(data_source_path):
    data = np.loadtxt(data_source_path, dtype=int)
    n, p = data.shape
    nu = n - 1


    return data, n, p, nu
    

In [9]:
data_7_15, n_7_15, p_7_15, nu_7_15 = load_data_7_15(DATA_SOURCE_PATH_7_15)

part I

In [10]:
def calc_H_0_input_7_15_1(data, p, nu):

    S = np.cov(data, rowvar=False)

    u = (p ** p)  * (np.linalg.det(S) / (np.trace(S) ** p)) 
    u_t = -(nu - (2 * (p**2) + p + 2) / (6 * p)) * np.log(u)

    eig_vals, _ = np.linalg.eig(S)
    eig_vals = eig_vals[:p]

    u = (p ** p)  * (np.prod(eig_vals) / (np.sum(eig_vals) ** p)) 
    u_t = - (nu - (2 * (p**2) + p + 2) / (6 * p)) * np.log(u)

    df = (0.5 * p * (p + 1) - 1)
    xi_critical = stats.chi2.ppf(1 - ALPHA, df=df)

    return u_t, xi_critical

In [11]:
u_t, xi_critical = calc_H_0_input_7_15_1(data_7_15, p_7_15, nu_7_15 )

In [12]:
check_H_0(u_t, xi_critical)

Accept H_0


part II

In [13]:
def calc_H_0_input_7_18_2(data, p, nu):
    C = np.asarray([
        [3/sqrt(12), -1/sqrt(12), -1/sqrt(12), -1/sqrt(12)],
        [0, 2/sqrt(6), -1/sqrt(6), -1/sqrt(6)],
        [0, 0, 1/sqrt(2), -1/sqrt(2)],
    ])
    S = np.cov(data, rowvar=False)
    S_ort = C @ S @ C.T
    p_ort = p - 1

    u = (p_ort ** p_ort)  * (np.linalg.det(S_ort) / (np.trace(S_ort) ** p_ort)) 
    u_t = - (nu - (2 * (p_ort**2) + p_ort + 2) / (6 * p_ort)) * np.log(u)

    df = (0.5 * p_ort * (p_ort + 1) - 1)
    xi_critical = stats.chi2.ppf(1 - ALPHA, df=df)

    return u_t, xi_critical

In [14]:
u_t, xi_critical = calc_H_0_input_7_18_2(data_7_15, p_7_15, nu_7_15) 

In [15]:
check_H_0(u_t, xi_critical)

Reject H_0


## 7.28

Test independence of $(y_1, y_2)$ and $(x_1, x_2)$ for the sons data in Table 3.7.

In [16]:
def load_data_7_28(data_source_path):
    data = np.loadtxt(data_source_path, dtype=int)
    k = 2
    n, p = data.shape

    son_a = data[:, :2]
    n_a, p_a = son_a.shape

    son_b = data[:, 2:]
    n_b, p_b = son_b.shape

    nu = min(n_a, n_b) - k

    S = np.cov(data, rowvar=False)
    s_xx = S[:2, :2]
    s_yy = S[-2:, -2:]

    L = np.linalg.det(S) / (np.linalg.det(s_xx) * np.linalg.det(s_yy))
    a2 = p**2 - (p_a ** 2 + p_b ** 2)
    a3 = p**3 - (p_a ** 3 + p_b ** 3)
    f = a2 * 0.5
    c = 1 - (2 * a3 + 3 * a2) / (12 * f * nu)
    L_t = -nu * c * np.log(L)

    xi_critical = stats.chi2.ppf(1 - ALPHA, df=f)

    return L_t, xi_critical

    

In [17]:
L_t, xi_critical = load_data_7_28(DATA_SOURCE_PATH_7_28)

In [18]:
check_H_0(L_t, xi_critical)

Accept H_0
